In [5]:
import tensorflow as tf
import random
import pathlib
import numpy as np
import math
import face_recognition as fr
import cv2
from PIL import Image
import dlib
AUTOTUNE = tf.data.experimental.AUTOTUNE


In [9]:
video_capture = cv2.VideoCapture(0)
ret, frame = video_capture.read()
print(frame.shape)
video_capture.release()


(480, 640, 3)


In [6]:
image = fr.load_image_file("datasets/lfw/Aaron_Eckhart/Aaron_Eckhart_0001.jpg")
Image.fromarray(image).show()
locations = fr.face_locations(image, model="cnn")
for face_location in locations:
    top, right, bottom, left = face_location
    print(top, right, bottom, left)
    face = image[top:bottom, left:right]
    Image.fromarray(face).show()

72 165 170 66
80 175 187 67


In [4]:
video_capture = cv2.VideoCapture(0)
Hillary_Clinton_image = fr.load_image_file("datasets\lfw\Hillary_Clinton\Hillary_Clinton_0001.jpg")
Hillary_Clinton_location = fr.face_locations(Hillary_Clinton_image, model="cnn")[0]
Hillary_Clinton_encoding = fr.face_encodings(Hillary_Clinton_image, num_jitters=1)[0]
known_face_encodings = [Hillary_Clinton_encoding]
known_face_names = ["Hillary Clinton"]

face_locations = []
face_encodings = []
face_names = []
process_this_frame = True


In [9]:
frame_rate = 10
frame_count = 0
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    
    if frame_count % frame_rate == 0:
        frame_count = 0
        
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = fr.face_locations(rgb_small_frame, model="cnn")
        face_encodings = fr.face_encodings(rgb_small_frame, face_locations,num_jitters=1)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = fr.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = fr.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    frame_count += 1

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

error: OpenCV(4.1.1) D:\Repo\opencv-4.1.1\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [192, 192])
    image /= 255.0  # normalize to [0,1] range

    return image


def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label

In [ ]:
#load data
data_root = pathlib.Path("datasets/lfw")
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
image_count = len(all_image_paths)
random.shuffle(all_image_paths)

label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index, name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]


In [ ]:
ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))
image_label_ds = ds.map(load_and_preprocess_from_path_label)
ds = image_label_ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
ds = ds.batch(64)
ds = ds.prefetch(buffer_size=AUTOTUNE)


In [ ]:
model_v1 = tf.keras.applications.MobileNetV2(input_shape=(192, 192, 3), include_top=False)
model_v1.trainable = False
model = tf.keras.Sequential([model_v1,
                             tf.keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dense(len(label_names), activation='softmax')])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])
print(model.summary())


In [ ]:
model.fit(ds, epochs=5, steps_per_epoch=10)

In [ ]:
for i in range(10):
    test_data = load_and_preprocess_image(all_image_paths[i])
    test_data = np.reshape(test_data,newshape=(1,192,192,3))
    
    print(np.argmax(model.predict(test_data)),all_image_labels[i])
